In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from datetime import datetime
from matplotlib import style

# Rebalancing
- Rebalancing시, 필요한 데이터 : Risky universe data, Cash universe data
- 전략 : 12개월 데이터를 통해 구한 Fast momentum 사용 / 12M window 를 통한 B,T(parameter) optimize

In [2]:
# %% 데이터 읽어오기
# 읽어오는 기간 : rebalancing 하는 달의 직전 달까지
raw_risky_ETF_data = pd.read_excel('Universe.xlsx', index_col = 0).iloc[:, :-3][:'2019-5']
raw_cash_ETF_data = pd.read_excel('Universe.xlsx', index_col = 0).iloc[:, -3:][:'2019-5']
#raw_kospi_data = pd.read_excel('Kospi_data.xlsx', index_col = 0)['2012-10-31':'2019-5']
raw_risky_ETF_data.dropna(inplace=True)
raw_cash_ETF_data.dropna(inplace=True)

In [3]:
# %% 월말 주가 뽑기
risky_ETF_data = raw_risky_ETF_data.resample('M').last()
cash_ETF_data = raw_cash_ETF_data.resample('M').last()
#kospi_data = raw_kospi_data.resample('M').last()

In [4]:
risky_ETF_data

,TIGER 미국S&P500선물(H),TIGER 유로스탁스50(합성 H),TIGER 일본니케이225,KODEX MSCI EM선물(H),TIGER 라틴35,KODEX 중국본토CSI300,TIGER 미국MSCI리츠(합성 H),TIGER 원유선물Enhanced(H),KODEX 골드선물(H),KODEX 미국채울트라30년선물(H),TIGER 단기선진하이일드(합성 H),ARIRANG 단기우량채권
2011-10-31,13090,7539.597798,5325.882824,7495.791386,5185,6758.090079,8590.899989,10100,13105,8321.482230,8154.218033,43048.467988
2011-11-30,12230,7366.408506,4997.752047,7395.245709,4755,6322.337430,8241.939105,10840,13145,8567.322770,8026.852272,43249.589976
2011-12-31,12890,7322.534306,5010.041101,7081.318298,4910,5881.595150,8581.422039,10920,11880,8829.510763,8190.228381,43368.339756
2012-01-31,13520,7638.978549,5215.743511,7832.092399,5335,6178.766472,9122.096002,10880,13265,8817.564548,8410.331680,43542.283097
2012-02-29,14240,7940.692692,5761.302848,8261.659343,5660,6604.715377,8998.667245,11650,13625,8615.107632,8564.919281,43677.758198
2012-03-31,14540,7830.596269,5974.802941,8042.097570,5570,6155.297660,9420.759129,11230,12720,8271.811123,8612.121831,43811.142635
2012-04-30,14650,7290.545337,5641.404580,7888.874414,5405,6584.706710,9678.279337,11265,12740,8647.173790,8685.967833,43965.015590
2012-05-31,13680,6697.895941,5061.816296,7022.909956,4955,6599.449938,9210.197855,9440,12010,9340.054290,8580.956430,44125.996807
2012-06-30,14045,7158.701472,5336.779434,7350.893472,4765,6172.121990,9687.649583,8530,12060,9221.849631,8711.769990,44259.799376
2012-07-31,14590,7351.520359,5152.076257,7357.119978,4955,5849.450901,9862.345433,9565,12490,9534.966227,8824.833770,44549.565566


In [4]:
# %% Momentum 계산 - index: 날짜 / columns : variable(ETF명), value(Momentum), bad_assets(bad asset 갯수)
## P_0 = 2012-10-31인 momentum은 2012-11-30을 index로 하는 데이터로 들어감

def VAA_Momentum(Universe):
    Momentum = (1 * (Universe / Universe.shift(12)) + 2 * (Universe / Universe.shift(6)) + 4 * (
            Universe / Universe.shift(3)) + 12 * (Universe / Universe.shift(1))) - 19
    tmp_mom = pd.DataFrame(Momentum.iloc[-1,:]).T
    tmp_mom.index = [Momentum.index[-1] + relativedelta(days=1) + relativedelta(months=1) - relativedelta(days=1)]
    Momentum = Momentum.shift(1)
    Momentum = pd.concat([Momentum, tmp_mom], axis = 0)
    Momentum.dropna(inplace=True)
    df = pd.melt(Momentum.reset_index(), id_vars='index')
    df = df.set_index('index')
    df['bad_assets'] = df['value'].apply(lambda x: np.where(x <= 0, 1, 0))
    return df

In [5]:
risky_ETF_momentum = VAA_Momentum(risky_ETF_data)
cash_ETF_momentum = VAA_Momentum(cash_ETF_data)

In [7]:
risky_ETF_momentum.tail()

,variable,value,bad_assets
index,,,
2019-02-28,ARIRANG 단기우량채권,0.087950,0
2019-03-31,ARIRANG 단기우량채권,0.072173,0
2019-04-30,ARIRANG 단기우량채권,0.091917,0
2019-05-31,ARIRANG 단기우량채권,0.080744,0
2019-06-30,ARIRANG 단기우량채권,0.086681,0


In [6]:
# %% 리턴 데이터프레임 구성 - columns : index(날짜), variable(ETF명), value(monthly return)

def Return(Universe):
    df = Universe.pct_change().dropna()
    df = pd.melt(df.reset_index(), id_vars='index')
    return df

In [7]:
risky_ETF_monthly_return = Return(risky_ETF_data)
cash_ETF_monthly_return = Return(cash_ETF_data)
#kospi_monthly_return = Return(kospi_data).set_index(['index']).drop(['variable'], axis=1)
#Kospi_daily_return = raw_kospi_data.pct_change().dropna()

In [10]:
risky_ETF_monthly_return.tail()

,index,variable,value
1087,2019-01-31,ARIRANG 단기우량채권,0.001936
1088,2019-02-28,ARIRANG 단기우량채권,0.001159
1089,2019-03-31,ARIRANG 단기우량채권,0.002316
1090,2019-04-30,ARIRANG 단기우량채권,0.001540
1091,2019-05-31,ARIRANG 단기우량채권,0.001826


In [8]:
# %% vaa portfolio monthly, daily 수익률
## Protection_threshold(B), Top(T)가 정해진 경우, VAA 전략으로 투자했을 때의 portfolio return
## Ex) momentum data = 2012-11-30 ~ 2019-03-31, return data = 2011-11-30 ~ 2019-02-28 넣으면?
##    return되는 data = 2012-11-30 ~ 2019-02-28 (2012-11-01 투자부터 2019-02-01 투자까지의 수익률이 나옴)
## Ex) momentum data : risky_ETF_momentum['2013-01':]으로 넣으면, 2013-01-01 투자부터의 수익률이 나옴
##    monthly_return data는 그대로 두고, momentum data의 날짜만 변경함으로써 원하는 기간의 수익률 구할 수 있음

def VAA_return(Risky_momentum, Cash_momentum, Risky_return, Cash_return, Protection_threshold, Top):
    # risky & cash proportion
    if Protection_threshold == 0:
        cash_ETF_portion = (Risky_momentum.groupby(['index'])['bad_assets'].sum() / 1)
        cash_ETF_portion.iloc[:] = 1
    elif Protection_threshold != 0:
        cash_ETF_portion = (Risky_momentum.groupby(['index'])['bad_assets'].sum() / Protection_threshold).map(
            lambda x: np.where(x >= 1, 1, x))
    risky_ETF_portion = 1 - cash_ETF_portion
    
    # risky_ETF_list : momentum 기준으로 상위 Top개 list 뽑아냄 (index(날짜), columns : variable(ETF명))
    risky_ETF_list = Risky_momentum.groupby(['index'], group_keys=False).apply(lambda x: x.nlargest(Top, 'value')).iloc[:,:1]
    cash_ETF_list = Cash_momentum.groupby(['index'], group_keys=False).apply(lambda x: x.nlargest(1, 'value')).iloc[:, :1]

    # list 기준으로 monthly return 뽑아냄 (columns : index(날짜), variable(ETF명), value(monthly return))
    risky_ETF_return = pd.merge(Risky_return, risky_ETF_list, on=['index', 'variable']).sort_values(
        by='index').reset_index(drop=True)
    cash_ETF_return = pd.merge(Cash_return, cash_ETF_list, on=['index', 'variable']).sort_values(
        by='index').reset_index(drop=True)

    # equally weight
    vaa_port_monthly_return = ((risky_ETF_return.groupby(['index'])['value'].mean() * risky_ETF_portion) + (
            cash_ETF_return.groupby(['index'])['value'].mean() * cash_ETF_portion)).dropna()

    return vaa_port_monthly_return

In [9]:
# %% 퍼포먼스 계산
## return series를 넣으면? 해당 return series의 performance 계산
## Cumulative Return, Annualized Return, Annualized Volatility, MDD, MDD duration, Sharpe_ratio, MAR_ratio, RAD
## 참고! RAD>=0 (always)

def performance(port_monthly_return):
    cum_return = ((port_monthly_return + 1).cumprod() - 1)

    # 전체 누적 수익률
    cum_return_last = cum_return.tail(1).iloc[0]

    # 연 평균 수익률
    yearly_return = pow(cum_return_last + 1, (1 / (len(port_monthly_return) / 12))) - 1
    # yearly_return_last_36m = pow(cum_return_last_36m_last + 1, (
    #         1 / (len(port_return_monthly[cum_return.index.strftime("%Y-%m").drop_duplicates()[-36]:]) / 12))) - 1

    # MDD
    hwm = [0]
    eq_idx = cum_return.index
    drawdown = pd.Series(index=eq_idx)
    duration = pd.Series(index=eq_idx)

    for t in range(1, len(eq_idx)):    # 1부터 len(eq_idx)-1까지
        cur_hwm = max(hwm[t - 1], float(cum_return.iloc[t]))
        hwm.append(cur_hwm)
        drawdown[t] = hwm[t] - float(cum_return.iloc[t])
        duration[t] = 0 if drawdown[t] == 0 else duration[t - 1] + 1

    MDD = drawdown.max()
    Duration = duration.max()    # Duration 조금 이상! 의미 없으므로 무시할 것.

    # sharpe
    yearly_std = port_monthly_return.std() * np.sqrt(12)
    sharpe = float(yearly_return) / float(yearly_std)

    # MAR ratio
    MAR = yearly_return / MDD if MDD != 0 else 0    # B,T optimize할 때, MDD=0인 경우 발생, 이 경우 MAR ration = 0으로 둠

    # RAD ratio
    RAD = yearly_return * (1 - (MDD / (1 - MDD))) if (yearly_return >= 0) & (MDD <= 0.5) else 0

    # performance
    performance_data = {'Cumulative Return': float(cum_return_last),
                        'Annualized Return': float(yearly_return),
                        'Annualized Volatility': float(yearly_std),
                        'MDD': MDD, 'MDD duration': Duration, 'Sharpe_ratio': sharpe, 'MAR_ratio': float(MAR),
                        'RAD': float(RAD)}

    total_performance = pd.Series(performance_data)

    return total_performance

In [10]:
# B,T optimize - RAD 기준 / UP 응용

def opt_B_T(Risky_momentum, Cash_momentum, Risky_return, Cash_return):

    # 각 (b,t)에 대한 performance
    opt_performance = pd.DataFrame()
    columns = []
    for b in range(1,7):
        for t in range(1,7):
            columns.append((b, t))
            VAA_port_return_ = VAA_return(Risky_momentum, Cash_momentum, Risky_return, Cash_return, b, t)
            VAA_performance = performance(VAA_port_return_)
            opt_performance = pd.concat([opt_performance, VAA_performance], axis=1, sort=True)
    opt_performance.columns = columns
    
    # B,T columns DataFrame
    x = np.arange(36)
    x_1 = x//6+1
    x_2 = x%6+1
    x_1 = pd.Series(x_1, index = columns)
    x_2 = pd.Series(x_2, index = columns)
    
    # Weighted DataFrame
    b = pd.DataFrame([opt_performance.loc['RAD']]).T
    c = pd.concat([b, b/(float(b.sum())), x_1, x_2], axis = 1)
    c.columns = ['RAD', 'RAD/sum(RAD)', 'B', 'T']
    c['B_'] = c['RAD/sum(RAD)']*c['B']
    c['T_'] = c['RAD/sum(RAD)']*c['T']
    
    # B, T optimize
    d = c.sum()
    B = round(float(d['B_']))
    T = round(float(d['T_']))
    
    if B > 6:
        B = 6
    if T > 6:
        T = 6
        
    if B == 0:
        B = 1
    if T == 0:
        T = 1
    
    return int(B), int(T)

In [11]:
# B,T optimize - RAD 기준 / UP 응용
# DataFrame 확인용

def opt_B_T_check(Risky_momentum, Cash_momentum, Risky_return, Cash_return):

    # 각 (b,t)에 대한 performance
    opt_performance = pd.DataFrame()
    columns = []
    for b in range(1,7):
        for t in range(1,7):
            columns.append((b, t))
            VAA_port_return_ = VAA_return(Risky_momentum, Cash_momentum, Risky_return, Cash_return, b, t)
            VAA_performance = performance(VAA_port_return_)
            opt_performance = pd.concat([opt_performance, VAA_performance], axis=1, sort=True)
    opt_performance.columns = columns
    
    # B,T columns DataFrame
    x = np.arange(36)
    x_1 = x//6+1
    x_2 = x%6+1
    x_1 = pd.Series(x_1, index = columns)
    x_2 = pd.Series(x_2, index = columns)
    
    # Weighted DataFrame
    b = pd.DataFrame([opt_performance.loc['RAD']]).T
    c = pd.concat([b, b/(float(b.sum())), x_1, x_2], axis = 1)
    c.columns = ['RAD', 'RAD/sum(RAD)', 'B', 'T']
    c['B_'] = c['RAD/sum(RAD)']*c['B']
    c['T_'] = c['RAD/sum(RAD)']*c['T']
    
    # B, T optimize
    d = c.sum()
    B = round(float(d['B_']))
    T = round(float(d['T_']))
    
    if B > 6:
        B = 6
    if T > 6:
        T = 6
        
    if B == 0:
        B = 1
    if T == 0:
        T = 1
    
    return c, int(B), int(T)

# Rebalancing 비중

In [15]:
# rebalancing 달 : 2019-06-01
r = relativedelta(months=1)

c = datetime(2019,7,1)               # 투자하는 달의 다음 달 1일로 작성할 것!!
d = c - relativedelta(days=1)         # 투자하는 달
d_ = c - r - relativedelta(days=1)    # 투자 직전 달
e = c - r*12 - relativedelta(days=1)  # 12개월

In [16]:
d

datetime.datetime(2019, 6, 30, 0, 0)

In [17]:
d_

datetime.datetime(2019, 5, 31, 0, 0)

In [18]:
e

datetime.datetime(2018, 6, 30, 0, 0)

In [19]:
# B,T 최적화 : 직전 12개월 기준
b_, t_ = opt_B_T(risky_ETF_momentum[e.strftime('%Y-%m'):d_.strftime('%Y-%m')], cash_ETF_momentum[e.strftime('%Y-%m'):d_.strftime('%Y-%m')],
                 risky_ETF_monthly_return, cash_ETF_monthly_return)

In [20]:
b_

3

In [21]:
t_

4

In [23]:
risky_ETF_momentum[d.strftime('%Y-%m')]

,variable,value,bad_assets
index,,,
2019-06-30,TIGER 미국S&P500선물(H),-0.699588,1
2019-06-30,TIGER 유로스탁스50(합성 H),-0.435248,1
2019-06-30,TIGER 일본니케이225,-0.165701,1
2019-06-30,KODEX MSCI EM선물(H),-1.244827,1
2019-06-30,TIGER 라틴35,-0.087931,1
2019-06-30,KODEX 중국본토CSI300,-0.305387,1
2019-06-30,TIGER 미국MSCI리츠(합성 H),0.261470,0
2019-06-30,TIGER 원유선물Enhanced(H),-1.773280,1
2019-06-30,KODEX 골드선물(H),0.024662,0


In [22]:
# risky, cash 비중 계산
if b_ == 0:
    cash_ETF_portion = (risky_ETF_momentum[d.strftime('%Y-%m')].groupby(['index'])['bad_assets'].sum() / 1)
    cash_ETF_portion.iloc[:] = 1
elif b_ != 0:
    cash_ETF_portion = (risky_ETF_momentum[d.strftime('%Y-%m')].groupby(['index'])['bad_assets'].sum() / b_).map(lambda x: np.where(x >= 1, 1, x))
risky_ETF_portion = 1 - cash_ETF_portion

In [24]:
cash_ETF_portion

index
2019-06-30    1.0
Name: bad_assets, dtype: float64

In [25]:
risky_ETF_portion

index
2019-06-30    0.0
Name: bad_assets, dtype: float64

In [26]:
# risky_ETF_list : momentum 기준으로 상위 Top개 list 뽑아냄 (index(날짜), columns : variable(ETF명))
risky_ETF_list = risky_ETF_momentum[d.strftime('%Y-%m')].groupby(['index'], group_keys=False).apply(lambda x: x.nlargest(t_, 'value')).iloc[:,:1]
cash_ETF_list = cash_ETF_momentum[d.strftime('%Y-%m')].groupby(['index'], group_keys=False).apply(lambda x: x.nlargest(1, 'value')).iloc[:, :1]

In [27]:
risky_ETF_list

,variable
index,
2019-06-30,KODEX 미국채울트라30년선물(H)
2019-06-30,TIGER 미국MSCI리츠(합성 H)
2019-06-30,TIGER 단기선진하이일드(합성 H)
2019-06-30,ARIRANG 단기우량채권


In [28]:
cash_ETF_list

,variable
index,
2019-06-30,KBSTAR 국채선물10년


In [29]:
# 투자할 ETF 및 비중 DataFrame
risky_ETF_list['weight'] = risky_ETF_portion[0] / (t_)
cash_ETF_list['weight'] = cash_ETF_portion[0]
ETF_list = pd.concat([risky_ETF_list, cash_ETF_list])

In [30]:
ETF_list

,variable,weight
index,,
2019-06-30,KODEX 미국채울트라30년선물(H),0.0
2019-06-30,TIGER 미국MSCI리츠(합성 H),0.0
2019-06-30,TIGER 단기선진하이일드(합성 H),0.0
2019-06-30,ARIRANG 단기우량채권,0.0
2019-06-30,KBSTAR 국채선물10년,1.0


In [31]:
ETF_list.to_excel('rebalancing %s.xlsx'%(d.strftime('%Y-%m')))

In [12]:
# rebalancing 달 : 2019-05-01
r = relativedelta(months=1)

c = datetime(2019,6,1)               # 투자하는 달의 다음 달 1일로 작성할 것!!
d = c - relativedelta(days=1)         # 투자하는 달
d_ = c - r - relativedelta(days=1)    # 투자 직전 달
e = c - r*12 - relativedelta(days=1)  # 12개월

In [13]:
d

datetime.datetime(2019, 5, 31, 0, 0)

In [14]:
d_

datetime.datetime(2019, 4, 30, 0, 0)

In [15]:
e

datetime.datetime(2018, 5, 31, 0, 0)

In [16]:
# B,T 최적화 : 직전 12개월 기준
b_, t_ = opt_B_T(risky_ETF_momentum[e.strftime('%Y-%m'):d_.strftime('%Y-%m')], cash_ETF_momentum[e.strftime('%Y-%m'):d_.strftime('%Y-%m')],
                 risky_ETF_monthly_return, cash_ETF_monthly_return)

In [17]:
b_

3

In [18]:
t_

4

In [19]:
risky_ETF_momentum[d.strftime('%Y-%m')]

,variable,value,bad_assets
index,,,
2019-05-31,TIGER 미국S&P500선물(H),1.149107,0
2019-05-31,TIGER 유로스탁스50(합성 H),1.320055,0
2019-05-31,TIGER 일본니케이225,1.409257,0
2019-05-31,KODEX MSCI EM선물(H),0.390330,0
2019-05-31,TIGER 라틴35,0.412394,0
2019-05-31,KODEX 중국본토CSI300,2.239071,0
2019-05-31,TIGER 미국MSCI리츠(합성 H),0.066726,0
2019-05-31,TIGER 원유선물Enhanced(H),0.689362,0
2019-05-31,KODEX 골드선물(H),-0.167481,1


In [20]:
# risky, cash 비중 계산
if b_ == 0:
    cash_ETF_portion = (risky_ETF_momentum[d.strftime('%Y-%m')].groupby(['index'])['bad_assets'].sum() / 1)
    cash_ETF_portion.iloc[:] = 1
elif b_ != 0:
    cash_ETF_portion = (risky_ETF_momentum[d.strftime('%Y-%m')].groupby(['index'])['bad_assets'].sum() / b_).map(lambda x: np.where(x >= 1, 1, x))
risky_ETF_portion = 1 - cash_ETF_portion

In [21]:
cash_ETF_portion

index
2019-05-31    0.333333
Name: bad_assets, dtype: float64

In [22]:
risky_ETF_portion

index
2019-05-31    0.666667
Name: bad_assets, dtype: float64

In [23]:
# risky_ETF_list : momentum 기준으로 상위 Top개 list 뽑아냄 (index(날짜), columns : variable(ETF명))
risky_ETF_list = risky_ETF_momentum[d.strftime('%Y-%m')].groupby(['index'], group_keys=False).apply(lambda x: x.nlargest(t_, 'value')).iloc[:,:1]
cash_ETF_list = cash_ETF_momentum[d.strftime('%Y-%m')].groupby(['index'], group_keys=False).apply(lambda x: x.nlargest(1, 'value')).iloc[:, :1]

In [24]:
risky_ETF_list

,variable
index,
2019-05-31,KODEX 중국본토CSI300
2019-05-31,TIGER 일본니케이225
2019-05-31,TIGER 유로스탁스50(합성 H)
2019-05-31,TIGER 미국S&P500선물(H)


In [25]:
cash_ETF_list

,variable
index,
2019-05-31,KBSTAR 국채선물10년


In [26]:
# 투자할 ETF 및 비중 DataFrame
risky_ETF_list['weight'] = risky_ETF_portion[0] / (t_)
cash_ETF_list['weight'] = cash_ETF_portion[0]
ETF_list = pd.concat([risky_ETF_list, cash_ETF_list])

In [27]:
ETF_list

,variable,weight
index,,
2019-05-31,KODEX 중국본토CSI300,0.166667
2019-05-31,TIGER 일본니케이225,0.166667
2019-05-31,TIGER 유로스탁스50(합성 H),0.166667
2019-05-31,TIGER 미국S&P500선물(H),0.166667
2019-05-31,KBSTAR 국채선물10년,0.333333


In [31]:
ETF_list.to_excel('rebalancing %s.xlsx'%(d.strftime('%Y-%m')))